# Estimación de ICI mediante histogramas de diagramas de constelación

## Librerías

In [1]:
import sofa

import polars as pl
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import tensorflow.keras as ker
import json
import os

from scipy.stats import multivariate_normal
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.colors import LogNorm

from sklearn.mixture import GaussianMixture
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_validate, KFold, train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow.keras import models, regularizers, Sequential, utils
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

from itertools import product
from collections import defaultdict

2023-08-14 14:58:40.445320: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-14 14:58:40.447368: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-14 14:58:40.484596: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-14 14:58:40.485327: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-14 14:58:41.108677: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

## Funciones globales

### Extraer datos y calcular los histogramas

In [2]:
# Special function to read the known data structure
def read_data(folder_rx):
    data = {}

    # Read root directory
    for folder in os.listdir(folder_rx):
        # Check name consistency for subdirectories 
        if folder.endswith("spacing"):
            # Extract "pretty" part of the name
            spacing = folder[:-8]
            data[spacing] = {}
            
            # Read each data file
            for file in os.listdir(f"{folder_rx}/{folder}"):
                # Check name consistency for data files
                if file.find("consY") != -1:
                    # Extract "pretty" part of the name
                    osnr = file.split("_")[2][5:-4]
                    
                    # Initialize if not created yet
                    if data[spacing].get(osnr) == None:
                        data[spacing][osnr] = {}
                    # Set data
                    csv_file_data = pl.read_csv(f"{folder_rx}/{folder}/{file}")
                    data[spacing][osnr] = csv_file_data
    return data

def split(a, n):
    k, m = divmod(len(a), n)
    return np.array([a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n)])

def plot_constellation_diagram(X, ax):
    ax.scatter(X.real, X.imag, alpha=0.5)
    ax.set_title("Constellation diagram")
    ax.set_xlabel("I")
    ax.set_ylabel("Q")
    
def calculate_gmm(data, gm_kwargs):
    return GaussianMixture(**gm_kwargs).fit(data)
    
def calculate_1d_histogram(X, bins):
    hist_y, hist_x = np.histogram(X.real, bins=bins)
    # Remove last bin edge
    hist_x = hist_x[:-1]
    
    return hist_x, hist_y

def plot_1d_histogram(X, ax):
    ax.hist(X, bins=bins, density=True, alpha=0.5, label="Calculated histogram")
    
def plot_gmm_1d(gm, limits):
    x = np.linspace(*limits, 1000)
    
    logprob = gm.score_samples(x.reshape(-1, 1))
    responsibilities = gm.predict_proba(x.reshape(-1, 1))
    pdf = np.exp(logprob)
    pdf_individual = responsibilities * pdf[:, np.newaxis]
    
    ax.plot(x, pdf_individual, '--', label="Adjusted histogram")

def plot_gmm_2d(gm, limits, ax):
    x = y = np.linspace(*limits)
    X, Y = np.meshgrid(x, y)
    Z = -gm.score_samples(np.array([X.ravel(), Y.ravel()]).T).reshape(X.shape)

    ax.contour(
        X, Y, Z,
        norm=LogNorm(vmin=1.0, vmax=1000.0), 
        levels=np.logspace(0, 3, 25), cmap="seismic"
    )
    
def calculate_3d_histogram(X, bins, limits, spacing, snr):
    hist, xedges, yedges = np.histogram2d(X.real, X.imag, bins=bins, range=[[*limits], [*limits]])

    # Define the extent
    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

    # Create the meshgrid for the surface plot, excluding the last edge
    x_mesh, y_mesh = np.meshgrid(xedges[:-1], yedges[:-1])
    
    return hist, x_mesh, y_mesh
    
def plot_3d_histogram(x_mesh, y_mesh, hist, ax):
    ax.plot_surface(x_mesh, y_mesh, hist.T, cmap="seismic", rstride=1, cstride=1, edgecolor="none")
    ax.set_title("3D Histogram")
    ax.set_xlabel("I")
    ax.set_ylabel("Q")

### Regresor

In [3]:
def calc_once(varname, fn, args):
    """ Calculate a variable only once. """
    if varname not in globals():
        return fn(**args)
    return eval(varname)


def estimation_model(
    layers_props_lst: list, loss_fn: ker.losses.Loss, input_dim: int
) -> ker.models.Sequential:
    """ Compile a sequential model for regression purposes. """
    model = ker.Sequential()
    # Hidden layers
    for i, layer_props in enumerate(layers_props_lst):
        if i == 0:
            model.add(ker.layers.Dense(input_dim=input_dim, **layer_props))
        else:
            model.add(ker.layers.Dense(**layer_props))
    # Regressor
    model.add(ker.layers.Dense(units=1, activation="linear"))
    
    model.compile(loss=loss_fn, optimizer="adam")
    
    return model


def estimation_crossvalidation(X, y, n_splits, layer_props, loss_fn, callbacks):
    """ Crossvalidation of an estimation network. """
    # Scores dict
    scores = {}
    scores["model"] = []
    scores["loss"] = []
    scores["mae"] = {"train": [], "test": []}
    scores["r2"] = {"train": [], "test": []}
    scores["rmse"] = {"train": [], "test": []}
    
    # K-fold crossvalidation
    kf = KFold(n_splits=n_splits, shuffle=True)

    for train_index, test_index in kf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Input variables standarizer
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test_kf = sc.transform(X_test)

        model = estimation_model(layer_props, loss_fn, X_train.shape[1])
        
        # Save test scalar loss
        if callbacks:
            loss = model.fit(
                X_train, y_train, epochs=5000, batch_size=64, callbacks=callbacks, verbose=0
            )
        else:
            loss = model.fit(X_train, y_train, epochs=5000, batch_size=64, verbose=0)
        print(f"Needed iterations: {len(loss.history['loss'])}")
        loss = loss.history["loss"]
        
        # Predict using train values
        predictions_train = model.predict(X_train, verbose=0)
        # Predict using test values
        predictions_test = model.predict(X_test_kf, verbose=0)

        # Dataframe for better visualization
        train_data_train = pl.DataFrame(
            {"ICI": [y_train], "Predicted ICI": [predictions_train]}
        )
        train_data_test = pl.DataFrame(
            {"ICI": [y_test], "Predicted ICI": [predictions_test]}
        )

        # MAE
        mae_score_train = mean_absolute_error(
            *train_data_train["ICI"], *train_data_train["Predicted ICI"]
        )
        mae_score_test = mean_absolute_error(
            *train_data_test["ICI"], *train_data_test["Predicted ICI"]
        )

        # R²
        r2_score_train = r2_score(
            *train_data_train["ICI"], *train_data_train["Predicted ICI"]
        )
        r2_score_test = r2_score(
            *train_data_test["ICI"], *train_data_test["Predicted ICI"]
        )
         
        # RMSE
        rmse_score_train = mean_squared_error(
            *train_data_train["ICI"], *train_data_train["Predicted ICI"],
            squared=False
        )
        rmse_score_test = mean_squared_error(
            *train_data_test["ICI"], *train_data_test["Predicted ICI"],
            squared=False
        )

        # Append to lists
        scores["model"].append(model)
        scores["loss"].append(loss)
        scores["mae"]["train"].append(mae_score_train)
        scores["mae"]["test"].append(mae_score_test)
        scores["r2"]["train"].append(r2_score_train)
        scores["r2"]["test"].append(r2_score_test)
        scores["rmse"]["train"].append(rmse_score_train)
        scores["rmse"]["test"].append(rmse_score_test)
        
    return scores


def test_estimation_model(data, n_splits, max_neurons, activations, 
                          use_osnr=True, loss_fn="mean_absolute_error"):
    """ Test a spectral spacing estimation model with given parameters. """
    var_n = 97 if use_osnr else 96

    # Split variables
    # Variables
    X = np.array(data[:, 0:var_n])
    # Tags
    y = np.array(data[:, -1])
    
    # Layer properties
    layer_props = [
        {"units": max_neurons // (2**i), "activation": activation}
        for i, activation in enumerate(activations)
    ]
    print(f"{layer_props}{' + OSNR' if use_osnr else ''}")
    callbacks = [
        EarlyStopping(monitor="loss", patience=30, mode="min", restore_best_weights=True)
    ]
    
    return estimation_crossvalidation(X, y, n_splits, layer_props, loss_fn, callbacks)

### Graficar resultados

In [4]:
def plot_loss(score, end=500):
    """ Plot loss evolution for each k-fold. """
    for k, loss in enumerate(score["loss"]):
        loss_length = len(loss.history["loss"])
        loss_values = loss.history["loss"][:end if loss_length > end else loss_length]
        epoch_values = range(end if loss_length > end else loss_length)
        plt.plot(epoch_values, loss_values, label=f"k = {k+1}")

        
def plot_losses(scores, scenario, cmp_fn, cmp_values=[], end=500, based_on_index=False):
    """ Plot loss graphics for each scenario. """
    # Handle default cmp_values
    if len(cmp_values) == 0:
        cmp_values = np.zeros(len(scores))
        
    fig_loss = plt.figure(figsize=(16, 3*len(scores)), layout="constrained")
    fig_loss.suptitle(f"{scenario} loss history", size="x-large")
    
    for index, (score, cmps) in enumerate(zip(scores, cmp_values)):
        plt.subplot(len(scores)//2, 2, index + 1)
        plot_loss(score, end=end)
        
        plt.title(cmp_fn(cmps) if not based_on_index else cmp_fn(index))
        
        plt.xlabel("Iteration")
        plt.ylabel("Loss")
        
        # Transparent white box black edge legend
        legend = plt.legend(loc="upper right", edgecolor="black")
        legend.get_frame().set_alpha(None)
        legend.get_frame().set_facecolor((1, 1, 1, 0.01))
        
        plt.grid(True)

def plot_scores(scores, x_values, scenario, score_names, data_type,
                label, xlabel, markers=[], colors=[], based_on_index=False,
                log=False, multiple_points=False, plot_train=True):
    fig_scores = plt.figure(figsize=(16, 6), layout="constrained")
    fig_scores.suptitle(f"{scenario} scores")
    i = 0
    for sn in score_names:
        ax = plt.subplot(1, len(score_names), i+1)
        for dt in data_type:
            if not plot_train and dt == "train":
                continue
            points = [np.average(score[sn][dt]) for score in scores]
            if not multiple_points:
                label_value = ""
                plt.scatter(x_values, points, marker=markers[0 if dt == "train" else 1],
                            label=f"{dt.title()} {label(i)}", s=100)
            else:
                label_value = ""
                points1 = points[::2]
                points2 = points[1::2]
                plt.scatter(x_values, points1, marker=markers[0 if dt == "train" else 1],
                            color=colors[0 if dt == "train" else 1],
                            label=f"{dt.title()} {label(0)}", s=100)
                plt.scatter(x_values, points2, marker=markers[2 if dt == "train" else 3],
                            color=colors[2 if dt == "train" else 3],
                            label=f"{dt.title()} {label(1)}", s=100)
            title = ""
            if sn == "r2":
                title = "R²"
            elif sn == "acc":
                title = "Accuracy"
            else:
                title = sn.upper()
            plt.title(title)
                
        plt.xlabel(xlabel)
        # Adjust logarithmic scale if requested
        if log:
            plt.xscale("log", base=2)
            
        # Make integer xticks if matches}
        if type(x_values[0]) == np.int64:
            ax.set_xticks(x_values)
            
        if type(x_values[0]) == str:
            ax.set_xticks(range(len(x_values)), x_values)
            
        # Shrink current axis by 20%
        box = ax.get_position()
        ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

        # Transparent white box black edge legend
        legend = plt.legend(loc="center left", bbox_to_anchor=(1, 0.5),
                            edgecolor="black")
        legend.get_frame().set_alpha(None)
        legend.get_frame().set_facecolor((1, 1, 1, 0.01))

        plt.grid(True)
        i += 1
    plt.show()
    
    
def plot_cm(scores, interval_lst):
    CM = np.array(scores.get("cm").get("test"))
    for n, interval in enumerate(interval_lst):
        result = np.zeros(CM[0][0].shape)
        for cm in CM:
            result = np.add(result, cm[n])
        result /= np.sum(result)
        disp = ConfusionMatrixDisplay(confusion_matrix=result, display_labels=["Positive", "Negative"])
        disp.plot(colorbar=False)
        lower_limit, upper_limit = interval 
        plt.title(f"Confusion matrix for class from {lower_limit} GHz up to {upper_limit} GHz")
        plt.show()

## Restaurar variables

In [5]:
%store -r histograms_hist
%store -r histograms_gmm
histograms = (histograms_hist, histograms_gmm)

## Leer datos

In [6]:
file_tx = "../../../Demodulation/Data/Processed/2x16QAM_16GBd.csv"
folder_rx = "../../../Demodulation/Data/Processed"

# Transmitted data
X_tx = np.array(pl.read_csv(file_tx))
X_txs = split(X_tx, 12)

# Read received data
data = read_data(folder_rx)

## Obtener histogramas

In [7]:
def get_histograms():
    spacings = ["15", "15.5", "16", "16.5", "17", "17.6", "18"]

    histograms_hist = defaultdict(lambda: defaultdict(list))
    histograms_gmm = defaultdict(lambda: defaultdict(list)) 
    bins = 128
    limits = [-5, 5]

    for spacing in spacings:
        X_rx = data[f"{spacing}GHz"]
        for snr in X_rx:
            # Extract data 
            X_ch = np.array(X_rx[snr])
            X_ch = X_ch[:, 0] + 1j*X_ch[:, 1]

            X_chs = split(X_ch, 12)

            for n, x_ch in enumerate(X_chs):
                # Calculate 2D GMM
                input_data = np.vstack((x_ch.real, x_ch.imag)).T
                gm_kwargs = {
                    "means_init": np.array(list(product([-3, -1, 1, 3], repeat=2))), 
                    "n_components": 16
                }
                gm_2d = calculate_gmm(input_data, gm_kwargs)

                # Calculate 3D histogram
                hist, x_mesh, y_mesh = calculate_3d_histogram(x_ch, bins, limits, spacing, snr)

                # Save 3D histogram
                histograms_hist[f"{spacing}GHz"][snr].append(hist)

                # Calculate I and Q histograms
                hist_x, hist_y = calculate_1d_histogram(x_ch.real, bins)
                input_data = np.repeat(hist_x, hist_y).reshape(-1, 1)
                gm_kwargs = {
                    "means_init": np.array([-3, -1, 1, 3]).reshape(4, 1), 
                    "n_components": 4
                }
                gm_i = calculate_gmm(input_data, gm_kwargs)

                # Q
                hist_x, hist_y = calculate_1d_histogram(x_ch.imag, bins)
                input_data = np.repeat(hist_x, hist_y).reshape(-1, 1)
                gm_kwargs = {
                    "means_init": np.array([-3, -1, 1, 3]).reshape(4, 1), 
                    "n_components": 4
                }
                gm_q = calculate_gmm(input_data, gm_kwargs)

                # Save gaussians
                histograms_gmm[f"{spacing}GHz"][snr].append([gm_2d, gm_i, gm_q])

    histograms_hist = dict(histograms_hist)
    histograms_gmm = dict(histograms_gmm)
    return histograms_hist, histograms_gmm

histograms = calc_once("histograms", get_histograms, {})
histograms_hist, histograms_gmm = histograms
%store histograms_hist
%store histograms_gmm

Stored 'histograms_hist' (dict)
Stored 'histograms_gmm' (dict)


In [8]:
def plot_histograms():
    spacings = ["15", "15.5", "16", "16.5", "17", "17.6", "18"]
    bins = 128
    limits = [-5, 5]
    
    for spacing in spacings:
        X_rx = data[f"{spacing}GHz"]
        for snr in X_rx:
            # Extract data 
            X_ch = np.array(X_rx[snr])
            X_ch = X_ch[:, 0] + 1j*X_ch[:, 1]
    
            plt.figure(figsize=(12, 12), layout="tight")
    
            # Plot constellation diagram
            ax = plt.subplot(2, 2, 1)
            plot_constellation_diagram(X_ch, ax)
            
            gm_2d = histograms_gmm.get(f"{spacing}GHz").get(snr)[0]
            
            # Plot 2D GMM
            plot_gmm_2d(gm_2d, limits, ax)
            ax.grid(True)
    
            # Calculate 3D histogram
            hist, x_mesh, y_mesh = calculate_3d_histogram(X_ch, bins, limits, spacing, snr)
            
            # Plot 3D histogram
            ax = plt.subplot(2, 2, 2, projection="3d")
            plot_3d_histogram(x_mesh, y_mesh, hist, ax)
    
            # Plot I and Q histograms separately
            # I
            ax = plt.subplot(2, 2, 3)
            plot_1d_histogram(X_ch.real, ax)
            
            hist_x, hist_y = calculate_1d_histogram(X_ch.real, bins)
            input_data = np.repeat(hist_x, hist_y).reshape(-1, 1)
            gm_kwargs = {
                "means_init": np.array([-3, -1, 1, 3]).reshape(4, 1), 
                "n_components": 4
            }
            gm_i = calculate_gmm(input_data, gm_kwargs)
            plot_gmm_1d(gm_i, limits)
    
            ax.set_title("I-Histogram")
            ax.set_xlabel("I")
            ax.grid(True)
    
            # Q
            ax = plt.subplot(2, 2, 4)
            plot_1d_histogram(X_ch.imag, ax)
            
            hist_x, hist_y = calculate_1d_histogram(X_ch.imag, bins)
            input_data = np.repeat(hist_x, hist_y).reshape(-1, 1)
            gm_kwargs = {
                "means_init": np.array([-3, -1, 1, 3]).reshape(4, 1), 
                "n_components": 4
            }
            gm_q = calculate_gmm(input_data, gm_kwargs)
            plot_gmm_1d(gm_q, limits)
            ax.set_title("Q-Histogram")
            ax.set_xlabel("Q")
            ax.grid(True)
        
            plt.suptitle(f"Plots for {snr} OSNR and {spacing}GHz of spacing")
    
            plt.show()

## Preparar datos

In [9]:
# Dataframe con 98 columnas
# 16 primeras para las medias
# 64 siguientes para los valores de las matrices de covarianza
# Penúltima para el valor del OSNR (dB)
# Última para el valor del espaciamiento (GHz)

df_dict = {f"col{n}": [] for n in range(98)}
data_list = []

# Iterate over the dictionary and populate the DataFrame
for spacing, osnr_dict in histograms_gmm.items():
    for osnr, gmm_list in osnr_dict.items():
        for n in range(12):
            gmm_2d = gmm_list[n][0]
            means = gmm_2d.means_.flatten()
            covariances = gmm_2d.covariances_.flatten()
            osnr_value = np.array([float(osnr[:-2])])
            spacing_value = np.array([float(spacing[:-3])])

            features = np.concatenate((means, covariances, osnr_value, spacing_value))
            row_dict = {f"col{n}": feature for n, feature in enumerate(features)}
            data_list.append(row_dict)

# Convert the list of dictionaries into a DataFrame
df = pl.DataFrame(data_list)

# Print the DataFrame
print(df)
df.write_json("histograms.json")

shape: (840, 98)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬──────────┬───────┬───────┐
│ col0      ┆ col1      ┆ col2      ┆ col3      ┆ … ┆ col94     ┆ col95    ┆ col96 ┆ col97 │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---      ┆ ---   ┆ ---   │
│ f64       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64      ┆ f64   ┆ f64   │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪══════════╪═══════╪═══════╡
│ -2.726003 ┆ -2.822036 ┆ -2.743914 ┆ -0.906839 ┆ … ┆ 0.006134  ┆ 0.247726 ┆ 23.0  ┆ 15.0  │
│ -2.707006 ┆ -2.735225 ┆ -2.773898 ┆ -0.827666 ┆ … ┆ -0.010791 ┆ 0.240321 ┆ 23.0  ┆ 15.0  │
│ -2.636324 ┆ -2.79119  ┆ -2.814175 ┆ -0.921746 ┆ … ┆ -0.014103 ┆ 0.244062 ┆ 23.0  ┆ 15.0  │
│ -2.735143 ┆ -2.759209 ┆ -2.714546 ┆ -0.934065 ┆ … ┆ 0.01472   ┆ 0.283967 ┆ 23.0  ┆ 15.0  │
│ …         ┆ …         ┆ …         ┆ …         ┆ … ┆ …         ┆ …        ┆ …     ┆ …     │
│ -2.685825 ┆ -2.698239 ┆ -2.743573 ┆ -0.789322 ┆ … ┆

## Evaluación de hiperparámetros

Se evaluará una combinación de parámetros:
- Número de neuronas máximas por capa (16, 64, 256, 1024)
- Número de capas (1, 2, 3)
- Combinación de funciones de activación (ReLu, tanh, sigmoid)

Los resultados tendrán la siguiente estructura:
```
{"xyz": {"n_neurons": {"osnr/wo_osnr": results}}}
```
Donde xyz serán las iniciales de las combinaciones de funciones de activación, n_neurons será el número de neuronas máximo, osnr/wo_osnr será la indicación de si se usó o no el OSNR como característica y results será el objeto resultante que contiene todos los resultados.

In [10]:
osnr_lst = ["osnr", "wo_osnr"]
max_neurons = [str(2**n) for n in range(3, 11)]
functs = ["relu", "tanh", "sigmoid"]
layers_n = [1, 2, 3]

combinations = [
    [list(subset) for subset in product(functs, repeat=n)]
    for n in layers_n
]

hidden_layers = [item for sublist in combinations for item in sublist]

[['relu'],
 ['tanh'],
 ['sigmoid'],
 ['relu', 'relu'],
 ['relu', 'tanh'],
 ['relu', 'sigmoid'],
 ['tanh', 'relu'],
 ['tanh', 'tanh'],
 ['tanh', 'sigmoid'],
 ['sigmoid', 'relu'],
 ['sigmoid', 'tanh'],
 ['sigmoid', 'sigmoid'],
 ['relu', 'relu', 'relu'],
 ['relu', 'relu', 'tanh'],
 ['relu', 'relu', 'sigmoid'],
 ['relu', 'tanh', 'relu'],
 ['relu', 'tanh', 'tanh'],
 ['relu', 'tanh', 'sigmoid'],
 ['relu', 'sigmoid', 'relu'],
 ['relu', 'sigmoid', 'tanh'],
 ['relu', 'sigmoid', 'sigmoid'],
 ['tanh', 'relu', 'relu'],
 ['tanh', 'relu', 'tanh'],
 ['tanh', 'relu', 'sigmoid'],
 ['tanh', 'tanh', 'relu'],
 ['tanh', 'tanh', 'tanh'],
 ['tanh', 'tanh', 'sigmoid'],
 ['tanh', 'sigmoid', 'relu'],
 ['tanh', 'sigmoid', 'tanh'],
 ['tanh', 'sigmoid', 'sigmoid'],
 ['sigmoid', 'relu', 'relu'],
 ['sigmoid', 'relu', 'tanh'],
 ['sigmoid', 'relu', 'sigmoid'],
 ['sigmoid', 'tanh', 'relu'],
 ['sigmoid', 'tanh', 'tanh'],
 ['sigmoid', 'tanh', 'sigmoid'],
 ['sigmoid', 'sigmoid', 'relu'],
 ['sigmoid', 'sigmoid', 'tanh'],
 

In [ ]:
try:
    histograms_reg_results = sofa.load_hdf5("histograms_reg_results.h5")
except:
    print("Error loading from file, creating a new dictionary")
    histograms_reg_results = defaultdict(defaultdict(defaultdict(defaultdict().copy).copy).copy)

# Evaluar
for activations in hidden_layers:
    for neurons in max_neurons:
        for osnr in osnr_lst:
            args = {"data": df, "n_splits": 5, "max_neurons": int(neurons), "activations": activations, "use_osnr": True if osnr == "osnr" else False}
            act_fn_name = "".join([s[0] for s in activations])
            if histograms_reg_results[act_fn_name][neurons][osnr] == defaultdict(): 
                # Get results
                results = test_estimation_model(**args)
                # Serialize model
                results["model"] = [utils.serialize_keras_object(model) for model in results["model"]]
                # Save serialized model for serialization
                histograms_reg_results[act_fn_name][neurons][osnr] = results
                # Save results with serialized model
                print("Saving results...")
                sofa.save_hdf5(histograms_reg_results, "histograms_reg_results.h5")
                print("Results saved!")
                

[{'units': 64, 'activation': 'sigmoid'}, {'units': 32, 'activation': 'sigmoid'}] + OSNR
Needed iterations: 856
Needed iterations: 837
Needed iterations: 581
Needed iterations: 638
Needed iterations: 1018
Saving results...
Results saved!
[{'units': 64, 'activation': 'sigmoid'}, {'units': 32, 'activation': 'sigmoid'}]
Needed iterations: 514
Needed iterations: 771
Needed iterations: 603
Needed iterations: 708
Needed iterations: 883
Saving results...
Results saved!
[{'units': 128, 'activation': 'sigmoid'}, {'units': 64, 'activation': 'sigmoid'}] + OSNR
Needed iterations: 466
Needed iterations: 608
Needed iterations: 538
Needed iterations: 633
Needed iterations: 654
Saving results...
Results saved!
[{'units': 128, 'activation': 'sigmoid'}, {'units': 64, 'activation': 'sigmoid'}]
Needed iterations: 798
Needed iterations: 519


In [ ]:
dd = defaultdict(defaultdict(defaultdict().copy).copy)
dd["r"]["1024"]
#histograms_reg_results["rr"]

## Gráficas

In [ ]:
classes_n = [2, 3, 4, 5, 6, 7]
OSNR = ["", "_woOSNR"]
classes_scores = {f"scores_histograms{osnr}":
                  [np.average(
                      eval(f"scores_histograms_{n}classes{osnr}").get(
                          "acc"
                      ).get(
                          "test"
                      )
                  ) for n in classes_n]
                  for osnr in OSNR
                 }
plt.figure(figsize=(8, 6), layout="constrained")
ax = plt.subplot(1, 1, 1)
for osnr in OSNR:
    OSNR_label = " with OSNR" if osnr == "" else " without OSNR"
    marker = "8"
    plt.plot(classes_n, classes_scores.get(f"scores_histograms{osnr}"),
             marker=marker, linestyle="--",
             label=OSNR_label)
plt.xlabel("Number of classes")
ax.set_xticks(classes_n)
plt.ylabel("Accuracy")
# Shrink current axis by 20%
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Transparent white box black edge legend
legend = plt.legend(loc="center left", bbox_to_anchor=(1, 0.5),
                    edgecolor="black")
legend.get_frame().set_alpha(None)
legend.get_frame().set_facecolor((1, 1, 1, 0.01))
plt.grid(True)

plt.savefig("hist_classes.svg", format="svg", transparent=True, bbox_inches="tight")